In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
#load the csv file and make the data frame
startup_df = pd.read_csv('../input/startup_funding.csv')

In [ ]:
#display the data frame
startup_df

In [ ]:
#display the how many rows and columns
print("The dataframe has {} rows and {} columns".format(startup_df.shape[0],startup_df.shape[1]))

In [ ]:
#display the data types of columns
startup_df.dtypes

In [ ]:
#display the informtion of dataframe
startup_df.info()

so from above we can see there are null values in some columns

In [ ]:
#display how many null values are there in each column
startup_df.apply(lambda x : sum(x.isnull()))

In [ ]:
#display graphically how many null values are there in each column 
sns.heatmap(startup_df.isnull(),cbar=0)
plt.show()

so from above we can see Remarks column has highest null values

In [ ]:
#fill the null values of CityLocation column with value NotSpecific
startup_df['CityLocation'] = startup_df['CityLocation'].fillna(value = 'NotSpecific')

In [ ]:
#fill the null values of IndustryVertical column with value Other
startup_df['IndustryVertical'] = startup_df['IndustryVertical'].fillna(value = 'Other')

In [ ]:
#display which CityLocation has more frequency or more startups
startup_df['CityLocation'].value_counts()

so from above we can see that it is consider Bangalore and bangalore different and some city location has two location separated by '/'.

In [ ]:
def convert_to_one_location(x):
    x = x.lower()
    if re.search('/',x):
        return(x.split('/')[0].strip())
    else:
        return(x.strip())
    
startup_df['CityLocation'] = startup_df['CityLocation'].apply(convert_to_one_location)

In [ ]:
startup_df['CityLocation'].value_counts()

so we make all city location column value to small letter and who is having two city location we are taking first one only. 

so from above we can see bangalore has highest number of startups

In [ ]:
startup_df['InvestmentType'].value_counts()

so from above we can see that it is consider Seed funding and SeedFunding as different and other column also like that 

In [ ]:
def align_investment(x):
    x = str(x).lower()
    x = x.replace(' ','')
    return(x)
startup_df['InvestmentType'] = startup_df['InvestmentType'].apply(align_investment)

In [ ]:
startup_df['InvestmentType'].value_counts()

so now we ave only four investment type and one null value is there

In [ ]:
# now copy this data frame and make new data frame and del unnecessary columns
new_startup_df = startup_df.copy()

In [ ]:
#display the new data frame
new_startup_df

so from above we can see Remarks column has highest null values we can remove that column and SNo column also we can remove

In [ ]:
del new_startup_df['Remarks']
del new_startup_df['SNo']

In [ ]:
new_startup_df

In [ ]:
#we have to make sure that all date are in proper format(i.e.,DD/MM/YYYY)
def count_date_proper(x):
    return(len(x.split('/')))
new_startup_df[new_startup_df['Date'].apply(count_date_proper) != 3]

so above what we are doing is we are splitting the date and after splitting length should be 3 if not three that means date is not properly formatted.

so from above we can see not properly formatted dates

In [ ]:
new_startup_df['Date'].replace('12/05.2015','12/05/2015',inplace=True)
new_startup_df['Date'].replace('13/04.2015','13/04/2015',inplace=True)
new_startup_df['Date'].replace('15/01.2015','15/01/2015',inplace=True)
new_startup_df['Date'].replace('22/01//2015','22/01/2015',inplace=True)

So now all dates are properly formatted

In [ ]:
#now we have to make date to time series
new_startup_df['Date'] = pd.to_datetime(new_startup_df['Date'],format='%d/%m/%Y')

In [ ]:
new_startup_df['Date'].dtypes

so from above we can see now dates are time series

In [ ]:
new_startup_df['AmountInUSD']

so from above we can see that amount is a string so we have to convert it into int and  we have to remove ',' 

In [ ]:
def convert_amount(x):
    if(re.search(',',x)):
        return(x.replace(',',''))
    else:
        return(x)
new_startup_df['AmountInUSD'] = new_startup_df[new_startup_df['AmountInUSD'].notnull()]['AmountInUSD'].apply(convert_amount).astype('int')

In [ ]:
#now we have to fill na values in AmountInUSD column with mean values
new_startup_df['AmountInUSD'] = new_startup_df['AmountInUSD'].fillna(value = np.mean(new_startup_df['AmountInUSD']))

In [ ]:
new_startup_df['AmountInUSD'] = new_startup_df['AmountInUSD'].astype('int')

In [ ]:
new_startup_df['InvestmentType'].replace('nan',np.NaN,inplace=True)

In [ ]:
new_startup_df['InvestmentType'].value_counts()

In [ ]:
#fill the null value of investment type with backward filling method
new_startup_df['InvestmentType'].fillna(method='bfill',inplace=True)

In [ ]:
#now we will calculate in each startup how many investors are there
def count_investors(x):
    if(re.search(',',x) and x!='empty'):
        return(len(x.split(',')))
    elif x != 'empty':
        return 1
    else:
        return -1
new_startup_df['NumberOfInvestors'] = new_startup_df['InvestorsName'].replace(np.NaN,'empty').apply(count_investors)

In [ ]:
#display which startup has maximum investors
new_startup_df[new_startup_df['NumberOfInvestors'] == new_startup_df['NumberOfInvestors'].max()]

so from above we can see Innov8 and MyChild App startup has maximum investors

In [ ]:
#fill na valus of InvestorsName with None
new_startup_df['InvestorsName'] = new_startup_df['InvestorsName'].fillna(value='None')

In [ ]:
#find unique Investors
Investors = []
investor_list = new_startup_df['InvestorsName'].apply(lambda x : x.lower().strip().split(','))

In [ ]:
for i in investor_list:
    for j in i:
        if(i!='none' or i!=''):
            Investors.append(j.strip())
unique_investors = list(set(Investors))

In [ ]:
new_startup_df.iloc[:,[1,2,3,4,6]] = new_startup_df.iloc[:,[1,2,3,4,6]].applymap(lambda x:x.lower().replace(' ','')if pd.notnull(x) is True else x)

In [ ]:
def check(x):
    if(pd.notnull(x)):
        return x.lower()
new_startup_df.iloc[:,3] = new_startup_df.iloc[:,3].apply(check)

In [ ]:
#display graphically how many null values are there
sns.heatmap(new_startup_df.isnull(),cbar=0)
plt.show()

In [ ]:
#make a list of unique startupname
uniquestartupname = list(new_startup_df['StartupName'].unique())

In [ ]:
#make a list of all startupname
startupname = list(new_startup_df['StartupName'])

In [ ]:
#remove ambigous records
for i in range(len(uniquestartupname)):
    for j in range(len(startupname)):
        if(re.search(uniquestartupname[i],startupname[j])):
            startupname[j] = uniquestartupname[i]

In [ ]:
#StartupName column without ambigous records
new_startup_df['StartupName'] = startupname

so from above we make records like ola.com as ola or flipkart.com as flipkart

In [ ]:
#display how many unique startup are there
len(new_startup_df['StartupName'].unique())

so 1792 unique startup are there

In [ ]:
#display 5 number summary of AmountInUSD column
new_startup_df['AmountInUSD'].describe().astype('int')

so from above we can see that standard deviation is very large that means data is more dispersed
and mean>median so it is right skewed

In [ ]:
#display top 10 funding startups
new_startup_df.groupby(by='StartupName')['AmountInUSD'].sum().sort_values(ascending =False).head(10)

so from above we can see paytm is highest funding startup and shopclues at 10 number

In [ ]:
#display top 10 startups which are having more investors
new_startup_df.groupby(by='StartupName').sum().sort_values(by='NumberOfInvestors',ascending=False).head(10)

so from above we can see ola has 46 investors